# Mixtures of MNLs

In [112]:
import gurobipy as gp

In [113]:
import numpy as np
import numpy.random as npr
N = 100 #5 #25 50 75 100
M = 1 #1 5 10 20
v = npr.rand((N+1), M)
for j in range(M):
    v[:, j] = v[:, j] / (100*v[0, j])
v = v[1:][:]
theta = list(np.random.dirichlet(np.ones(M), size=1)[0])
print(v)

[[0.00652151]
 [0.00606862]
 [0.00091226]
 [0.00625337]
 [0.01132393]
 [0.00304385]
 [0.00288363]
 [0.00082709]
 [0.00953737]
 [0.01123666]
 [0.00188687]
 [0.00907776]
 [0.00548826]
 [0.01037978]
 [0.00288436]
 [0.003294  ]
 [0.01030424]
 [0.01095349]
 [0.00565145]
 [0.00281234]
 [0.00954093]
 [0.00403193]
 [0.00755284]
 [0.00985225]
 [0.00439469]
 [0.00054821]
 [0.00938306]
 [0.00344867]
 [0.00689834]
 [0.01222964]
 [0.00236676]
 [0.00733709]
 [0.00812868]
 [0.00715188]
 [0.00735136]
 [0.00042738]
 [0.0095679 ]
 [0.00692732]
 [0.01014186]
 [0.00485272]
 [0.00946111]
 [0.0045196 ]
 [0.01014498]
 [0.00858506]
 [0.01074525]
 [0.01086842]
 [0.00420823]
 [0.01123622]
 [0.00794309]
 [0.0006142 ]
 [0.00734029]
 [0.00263383]
 [0.00511478]
 [0.00166758]
 [0.00734188]
 [0.00333886]
 [0.00128851]
 [0.01039251]
 [0.00112492]
 [0.00912345]
 [0.00039812]
 [0.01095135]
 [0.01020085]
 [0.00035989]
 [0.01187984]
 [0.00210018]
 [0.0072526 ]
 [0.01009345]
 [0.00549773]
 [0.00728693]
 [0.00798664]
 [0.00

In [114]:
def IP_mmnl(p):
    global v, N
    global theta, M

    #create a new model
    model = gp.Model("IP_mmnl")

    # create decision variables and store them in the arrays z, x, y
    z = model.addVars(M, vtype=gp.GRB.CONTINUOUS, name="z")
    x = model.addVars(N, M, vtype=gp.GRB.CONTINUOUS, lb=0, name="x")
    y = model.addVars(N, vtype=gp.GRB.BINARY, name="y")
    model.update()

    # set objective function
    objExpr = gp.LinExpr()
    for j in range(M):
        objExpr += theta[j] * z[j]
    model.setObjective(objExpr, gp.GRB.MAXIMIZE)

    B = max(p) + 1 
    # add constraints
    for j in range(M):
        myConstr = gp.LinExpr()
        for i in range(N):
            myConstr += v[i][j] * x[i, j]
            model.addConstr(lhs = x[i, j], sense = gp.GRB.LESS_EQUAL, rhs = y[i]*B, name = "x_1st_constraint_" + str(i) + '_' + str(j))
            model.addConstr(lhs = x[i, j], sense = gp.GRB.GREATER_EQUAL, rhs = p[i] - z[j] - (1-y[i])*B, name = "x_2nd_constraint_" + str(i) + '_' + str(j))
            model.addConstr(lhs = x[i, j], sense = gp.GRB.LESS_EQUAL, rhs = p[i] - z[j] + (1-y[i])*B, name = "x_3rd_constraint_" + str(i) + '_' + str(j))
        model.addConstr(lhs = z[j], sense = gp.GRB.LESS_EQUAL, rhs = myConstr, name = "z_" + str(j))

    # write model 
    model.write("IP_mmnl.lp")

    # solve model
    model.Params.OutputFlag = 0
    model.optimize()
    
    bestS = []
    profit = float(model.objVal)
    for i in range(N):
        if y[i].x == 1:
            bestS.append(i)
    bestS = sorted(bestS)
    return profit, bestS

In [115]:
def profit_mmnl(p, S):
    global v
    global theta, M
    pi = 0
    for j in range(M):
        Vj_S = 0
        num = 0
        for i in S:
            num += p[i]*v[i][j]
            Vj_S += v[i][j]
        pi += theta[j]*num/(1+Vj_S)
    return pi

In [116]:
def brute_force(p):
    global N
    # compute the profit of each assortment
    best_profit = 0
    best_S = []
    for i in range(2**N):
        binary = np.binary_repr(i, width=N)
        assortment = [int(x) for x in binary]
        S = []
        for j in range(N):
            if assortment[j] == 1:
                S.append(j)
        if best_profit < profit_mmnl(p, S):
            best_profit = profit_mmnl(p, S)
            best_S = S
    return best_profit, best_S

In [117]:
def nested_by_price_mmnl(p):
    global N
    profits = [0 for i in range(N)]
    S = []
    for i in range(N):
        S.append(i)
        profits[i] = profit_mmnl(p, S)
    best_S = [i for i in range(np.argmax(profits)+1)]
    return max(profits), best_S

In [118]:
def find_best_option_mmnl(p, S):
    global v, N
    current_best = profit_mmnl(p, S)
    bestS = S
    flag = 0
    N_S = [k for k in range(N) if k not in S]
    for k in N_S:
        newS = S + [k]
        if profit_mmnl(p, newS) >= current_best:
            flag = 1
            bestS = newS
            current_best = profit_mmnl(p, newS)
    return current_best, bestS, flag

def greedy_mmnl(p):
    global v, N
    max_profit = -1
    best_S = []
    S = []
    flag = 1
    while flag==1:
        max_profit, S, flag = find_best_option_mmnl(p, S)
    best_S = sorted(S)
    return max_profit, best_S

In [119]:
import matplotlib.pyplot as plt
import math
import statistics as st
import time

In [120]:
n_simulations = 100
avg_price = 1
std_price = 2
#compute prices
psim = [0 for k in range(n_simulations)]
for k in range(n_simulations):
    psim[k] = abs(npr.normal(avg_price, std_price, N))
    # p[i] = npr.uniform(0, 2*avg_price, N)
    # p[i] = npr.exponential(avg_price, N)
    psim[k] = sorted(psim[k], reverse=True)
print(psim)


[[8.385022953413756, 6.388120806119616, 6.14915630957345, 4.9163931302330175, 4.7692244715835965, 4.748624094752053, 4.484588051983211, 4.478643549152832, 4.453225526162404, 3.9591716432888653, 3.893377548080766, 3.8550931456550184, 3.5198243909073157, 3.4622502780876374, 3.319833289704986, 3.2709969079484074, 3.223893969265017, 3.181811667342755, 3.1421888225224786, 3.039950880966869, 3.02342608595782, 2.868251738820903, 2.8437004128244574, 2.8430758353903007, 2.7773325554342385, 2.761394977151605, 2.607976429916051, 2.5772178756078024, 2.4295971721287937, 2.4230630797401025, 2.35001359388587, 2.339221282329941, 2.27486690337411, 2.2063420388484563, 2.2046927396878035, 1.9959408624500101, 1.9128229216284873, 1.9079775610295275, 1.8701379212604097, 1.8610451607886256, 1.7960801932085924, 1.6423901804493943, 1.624601051931826, 1.5906635692581514, 1.5645051548630176, 1.4165635579363718, 1.388573856822042, 1.371792270767646, 1.367676904166119, 1.337908354220502, 1.296312959869773, 1.19341

In [121]:
# problem solved using the MIP formulation
profits_IP = []
times_IP = []
bestS_IP = []
for k in range(n_simulations):
    start_time = time.time()
    profit, bestS = IP_mmnl(psim[k])
    end_time = time.time()
    profits_IP.append(profit)
    times_IP.append(end_time-start_time)
    bestS_IP.append(bestS)
avg_time_IP = st.mean(times_IP)

In [122]:
# problem solved using the nested formulation
profits_nested = []
times_nested = []
bestS_nested = []
for k in range(n_simulations):
    start_time = time.time()
    profit, bestS = nested_by_price_mmnl(psim[k])
    end_time = time.time()
    profits_nested.append(profit)
    times_nested.append(end_time-start_time)
    bestS_nested.append(bestS)
avg_time_nested = st.mean(times_nested)

In [123]:
# problem solved using the greedy formulation
profits_greedy = []
times_greedy = []
bestS_greedy = []
for k in range(n_simulations):
    start_time = time.time()
    profit, bestS = greedy_mmnl(psim[k])
    end_time = time.time()
    profits_greedy.append(profit)
    times_greedy.append(end_time-start_time)
    bestS_greedy.append(bestS)
avg_time_greedy = st.mean(times_greedy)

In [124]:
# counting how many instances are different
count_nested = 0
different_results_nested = []
count_greedy = 0
different_results_greedy = []
for k in range(n_simulations):
    if bestS_IP[k] != bestS_greedy[k]:
        count_greedy += 100/n_simulations
        different_results_greedy.append(100*profits_greedy[k]/profits_IP[k])
    print(bestS_IP[k], bestS_greedy[k])
    # if bestS_IP[k] != bestS_nested[k]:
    #    count_nested += 100/n_simulations
    #   different_results_nested.append(100*profits_nested[k]/profits_IP[k])
    # print(bestS_IP, bestS_nested)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64] [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78] [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 

In [125]:
if different_results_greedy != []:
  print('The greedy algorithm is different from the MIP in ' + str(count_greedy) + ' of the instances' , \
        'with an average ratio of ' + str(st.mean(different_results_greedy)) + '%', \
              'and a minimum ratio of ' + str(min(different_results_greedy)) + '%', \
                'and an average time of ' + str(avg_time_greedy) + ' seconds')
if different_results_nested != []:
  print('The nested algorithm is different from the MIP in ' + str(count_nested) + ' of the instances', \
        'with an average ratio of ' + str(st.mean(different_results_nested)) + '%', \
              'and a minimum ratio of ' + str(min(different_results_nested)) + '%', 
                'and an average time of ' + str(avg_time_nested) + ' seconds')

The greedy algorithm is different from the MIP in 5.0 of the instances with an average ratio of 100.0024190846498% and a minimum ratio of 99.99999999327122% and an average time of 0.10563206672668457 seconds


In [126]:
# brute_force(psim[0])
